# **-------------------------------TRABALHO CCBA----------------------------------**

- David Carvalho, nº2242131
- Lígia Carteado Mena, nº2242194
- Rui Filipe Parada, nº2211025

### **NOTEBOOK DE PREVISÃO EM TEMPO REAL**

Correr código após fazer deploy do endpoint no notebook principal do projeto.

In [1]:
### PREVISÃO EM TEMPO REAL

# ---------------- IMPORTS NECESSÁRIOS ----------------
import pandas as pd
import joblib
from sagemaker.predictor import Predictor
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import JSONDeserializer

# ---------------- INICIALIZAÇÃO ----------------
# Nome do endpoint (substitui pelo nome real se estiver diferente)
ENDPOINT_NAME = 'sagemaker-xgboost-2025-06-12-15-38-19-254'

# Carrega os objetos
encoder = joblib.load('encoder_ccba.pkl')         # ColumnTransformer com OneHotEncoder e Scaler
label_encoder = joblib.load('label_encoder_ccba.pkl')  # LabelEncoder para o target

# Inicializa o predictor do SageMaker
xgb_predictor = Predictor(
    endpoint_name=ENDPOINT_NAME,
    serializer=CSVSerializer(),
    deserializer=JSONDeserializer()
)

# ---------------- INPUT DE TESTE ----------------
# Exemplo de novo smartphone para prever a faixa de preço
novo_smartphone = pd.DataFrame([{
    'Brand': 'Samsung',
    'Model': 'Galaxy M13',
    'RAM': 4,
    'Storage': 128,
    'Color': 'Black'
}])

# ---------------- TRANSFORMAÇÃO ----------------
# Aplicar transformações (encoder: ColumnTransformer com OneHot + Scaler)
X_encoded = encoder.transform(novo_smartphone)

# Serializa para CSV (modelo espera CSV como string, sem header)
csv_input = ','.join(map(str, X_encoded[0]))

# ---------------- PREVISÃO ----------------
# Obter resposta do endpoint
resposta = xgb_predictor.predict(csv_input)

# ---------------- INTERPRETAÇÃO ----------------
# Trata diferentes formatos de resposta
if isinstance(resposta, (float, int)):
    predicted_label = int(resposta)
    probability = None
elif isinstance(resposta, dict):
    predicted_label = resposta.get('predicted_label', None)
    probability = resposta.get('probability', 0)
else:
    predicted_label = None
    probability = 0
    print("⚠️ Resposta inesperada do endpoint:", resposta)

# Decodificar rótulo, se aplicável
if predicted_label is not None:
    decoded_label = label_encoder.inverse_transform([predicted_label])[0]
else:
    decoded_label = 'Desconhecido'

# ---------------- INTERVALO DA GAMA ----------------
def intervalo_preco(categoria):
    categoria = categoria.lower()
    if categoria == 'baixo':
        return 'menos de 200€'
    elif categoria == 'médio':
        return 'entre 200€ e 499€'
    elif categoria == 'alto':
        return 'entre 450€ e 949€'
    elif categoria == 'muito alto':
        return '950€ ou mais'
    else:
        return 'intervalo desconhecido'

# Mostrar resultado
print(f"📱 Gama de preço prevista: {decoded_label.upper()} ({intervalo_preco(decoded_label)})")
if probability:
    print(f"🤖 Confiança do modelo: {probability:.0%}")


/opt/conda/lib/python3.12/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
📱 Gama de preço prevista: MÉDIO (entre 200€ e 499€)
